In [ ]:
from litellm import completion
import gradio as gr
import json

OLLAMA_MODEL_NAME = 'ollama_chat/gpt-oss'

In [ ]:
SYSTEM_PROMT = """
    You are an AI assistant for a chip shop. 
    Ask customers what type of chips they want and how many they need. 
    Check whether the chips are available, and if so, provide the total price. 
    If you are missing any price or availability information, you must NOT assume or infer it. 
    You must answer: I don't know or Information unavailable.
    Keep responses short and accurate.

    Example of BAD behavior: assuming a price for an unknown chips type.
    Example of GOOD behavior: saying I don't know the price of garlic chips when no tool result exists.
"""

In [ ]:
pricy = {
    "onion": 30,
    "banana": 15,
    "potato": 20,
    "corn": 25
}

def get_chips_price(chips_type: str):
    price = pricy.get(chips_type.lower(), None)
    if price is None:
        return "We don't have this chips at the moment"
    return f"One packet of {chips_type} chips cost {price}rs"

In [ ]:
get_chips_price_definition = {
    "name": "get_chips_price",
    "description": "returns the price of specified type of chips",
    "parameters": {
        "type": "object",
        "properties": {
            "chips_type": {
                "type": "string",
                "description": "type of chips, only flavour"
            }
        },
        "required": ["chips_type"]
    }
}

tools = [
    {"type": "function", "function": get_chips_price_definition}
]

In [ ]:
def chat(message, history):

    system = {"role":"system", "content": SYSTEM_PROMT}
    query = {"role": "user", "content": message}
    history = [{"role": h['role'], "content": h['content']} for h in history]
    messages = [system]
    messages.extend(history)
    messages.append(query)
    response = completion(
        model = OLLAMA_MODEL_NAME, 
        messages = messages, 
        tools = tools, 
        tool_choice = 'auto',
        reasoning_effort = 'low'
    )

    print(response.choices[0])

    while response.choices[0].message.tool_calls:
        tool_responses = []
        for tool_call in response.choices[0].message.tool_calls:
            if tool_call.function.name == 'get_chips_price':
                function_arguments = json.loads(tool_call.function.arguments)
                chips_type = function_arguments['chips_type']
                result = get_chips_price(chips_type)
                tool_responses.append({
                    "role": "tool",
                    "content": result,
                    "tool_call_id": tool_call.id
                })
        
        messages.extend(tool_responses)

        print(messages)

        response = completion(
            model = OLLAMA_MODEL_NAME,
            messages = messages,
            tools = tools,
            tool_choice = 'auto',
            reasoning_effort = 'low'
        )

        print(response.choices[0])
        
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn = chat, type = 'messages').launch()